## Jリーグ観客数予測

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

## ファイル読み込み

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submit.csv', header=None)

In [3]:
train.head()

,id,y,year,stage,match,gameday,time,home,away,stadium,tv
0,13994,18250,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合
1,13995,24316,2012,Ｊ１,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋
2,13996,17066,2012,Ｊ１,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪
3,13997,29603,2012,Ｊ１,第１節第１日,03/10(土),14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,スカパー／ｅ２／スカパー光／ＮＨＫ広島
4,13998,25353,2012,Ｊ１,第１節第１日,03/10(土),14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,スカパー／ｅ２／スカパー光（スカイ・Ａ ｓｐｏｒｔｓ＋）／ＮＨＫ札幌


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1721 entries, 0 to 1720
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       1721 non-null   int64 
 1   y        1721 non-null   int64 
 2   year     1721 non-null   int64 
 3   stage    1721 non-null   object
 4   match    1721 non-null   object
 5   gameday  1721 non-null   object
 6   time     1721 non-null   object
 7   home     1721 non-null   object
 8   away     1721 non-null   object
 9   stadium  1721 non-null   object
 10  tv       1721 non-null   object
dtypes: int64(3), object(8)
memory usage: 148.0+ KB


In [5]:
test.head()

,id,year,stage,match,gameday,time,home,away,stadium,tv
0,15822,2014,Ｊ１,第１８節第１日,08/02(土),19:04,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス
1,15823,2014,Ｊ１,第１８節第１日,08/02(土),18:34,鹿島アントラーズ,サンフレッチェ広島,県立カシマサッカースタジアム,スカパー！／スカパー！プレミアムサービス
2,15824,2014,Ｊ１,第１８節第１日,08/02(土),19:04,浦和レッズ,ヴィッセル神戸,埼玉スタジアム２００２,スカパー！／スカパー！プレミアムサービス／ＮＨＫ ＢＳ１／テレ玉
3,15825,2014,Ｊ１,第１８節第１日,08/02(土),19:03,柏レイソル,川崎フロンターレ,日立柏サッカー場,スカパー！／スカパー！プレミアムサービス
4,15827,2014,Ｊ１,第１８節第１日,08/02(土),19:03,アルビレックス新潟,セレッソ大阪,デンカビッグスワンスタジアム,スカパー！／スカパー！プレミアムサービス


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313 entries, 0 to 312
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       313 non-null    int64 
 1   year     313 non-null    int64 
 2   stage    313 non-null    object
 3   match    313 non-null    object
 4   gameday  313 non-null    object
 5   time     313 non-null    object
 6   home     313 non-null    object
 7   away     313 non-null    object
 8   stadium  313 non-null    object
 9   tv       313 non-null    object
dtypes: int64(2), object(8)
memory usage: 24.6+ KB


In [7]:
sample.head(5)

,0,1
0,15822,18973.907177
1,15823,18895.017270
2,15824,20022.365635
3,15825,18853.916315
4,15827,18773.922407


In [8]:
train.describe()

,id,y,year
count,1721.000000,1721.000000,1721.000000
mean,15045.691458,10662.395119,2012.815224
std,648.205749,8106.877159,0.757613
min,13994.000000,0.000000,2012.000000
25%,14474.000000,4750.000000,2012.000000
50%,15044.000000,8650.000000,2013.000000
75%,15528.000000,13431.000000,2013.000000
max,16237.000000,54905.000000,2014.000000


## ステージを数値へ置き換え

In [9]:
train['stage'][train['stage'].str.contains('１') == True] = 1
train['stage'][train['stage'].str.contains('２') == True] = 2

test['stage'][test['stage'].str.contains('１') == True] = 1
test['stage'][test['stage'].str.contains('２') == True] = 2

/var/folders/p9/g996fg895cs0zrq0v3h2273m0000gn/T/ipykernel_40584/731220297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['stage'][train['stage'].str.contains('１') == True] = 1
/var/folders/p9/g996fg895cs0zrq0v3h2273m0000gn/T/ipykernel_40584/731220297.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['stage'][train['stage'].str.contains('２') == True] = 2
/var/folders/p9/g996fg895cs0zrq0v3h2273m0000gn/T/ipykernel_40584/731220297.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [10]:
train['stage']

0       1
1       1
2       1
3       1
4       1
       ..
1716    2
1717    2
1718    2
1719    2
1720    2
Name: stage, Length: 1721, dtype: object

In [11]:
train['stage'].value_counts()

2    1046
1     675
Name: stage, dtype: int64

In [12]:
test['stage'].value_counts()

2    174
1    139
Name: stage, dtype: int64

In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313 entries, 0 to 312
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       313 non-null    int64 
 1   year     313 non-null    int64 
 2   stage    313 non-null    object
 3   match    313 non-null    object
 4   gameday  313 non-null    object
 5   time     313 non-null    object
 6   home     313 non-null    object
 7   away     313 non-null    object
 8   stadium  313 non-null    object
 9   tv       313 non-null    object
dtypes: int64(2), object(8)
memory usage: 24.6+ KB


## 試合日を月、日、曜日に分解

In [14]:
train.head(3)

,id,y,year,stage,match,gameday,time,home,away,stadium,tv
0,13994,18250,2012,1,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合
1,13995,24316,2012,1,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋
2,13996,17066,2012,1,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪


In [15]:
buf1 = train['gameday'].apply(lambda x: x.split('(')[1])
week = buf1.apply(lambda x: x.split(')')[0])
buf1 = train['gameday'].apply(lambda x: x.split('/')[0])
month = buf1.astype(int)
buf1 = train['gameday'].apply(lambda x: x.split('/')[1])
day = buf1.apply(lambda x: x.split('(')[0]).astype(int)

train = train.drop('gameday', axis=1)
train['month'] = month
train['day'] = day
train['week'] = week

In [16]:
train.head()

,id,y,year,stage,match,time,home,away,stadium,tv,month,day,week
0,13994,18250,2012,1,第１節第１日,14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合,3,10,土
1,13995,24316,2012,1,第１節第１日,14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋,3,10,土
2,13996,17066,2012,1,第１節第１日,14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪,3,10,土
3,13997,29603,2012,1,第１節第１日,14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,スカパー／ｅ２／スカパー光／ＮＨＫ広島,3,10,土
4,13998,25353,2012,1,第１節第１日,14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,スカパー／ｅ２／スカパー光（スカイ・Ａ ｓｐｏｒｔｓ＋）／ＮＨＫ札幌,3,10,土


In [17]:
buf1 = test['gameday'].apply(lambda x: x.split('(')[1])
week = buf1.apply(lambda x: x.split(')')[0])
buf1 = test['gameday'].apply(lambda x: x.split('/')[0])
month = buf1.astype(int)
buf1 = test['gameday'].apply(lambda x: x.split('/')[1])
day = buf1.apply(lambda x: x.split('(')[0]).astype(int)

test = test.drop('gameday', axis=1)
test['month'] = month
test['day'] = day
test['week'] = week

## 節を数値に置き換え

In [18]:
test.head()

,id,year,stage,match,time,home,away,stadium,tv,month,day,week
0,15822,2014,1,第１８節第１日,19:04,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス,8,2,土
1,15823,2014,1,第１８節第１日,18:34,鹿島アントラーズ,サンフレッチェ広島,県立カシマサッカースタジアム,スカパー！／スカパー！プレミアムサービス,8,2,土
2,15824,2014,1,第１８節第１日,19:04,浦和レッズ,ヴィッセル神戸,埼玉スタジアム２００２,スカパー！／スカパー！プレミアムサービス／ＮＨＫ ＢＳ１／テレ玉,8,2,土
3,15825,2014,1,第１８節第１日,19:03,柏レイソル,川崎フロンターレ,日立柏サッカー場,スカパー！／スカパー！プレミアムサービス,8,2,土
4,15827,2014,1,第１８節第１日,19:03,アルビレックス新潟,セレッソ大阪,デンカビッグスワンスタジアム,スカパー！／スカパー！プレミアムサービス,8,2,土


In [19]:
def get_match(x):
    b = x[1:3]
    if b[1] == "節":
        b = b[0]
    return int(b)

In [20]:
train['match'] = train['match'].apply(lambda x: get_match(x))

In [21]:
test['match'] = test['match'].apply(lambda x: get_match(x))

## 地上波の有無を取得し、新カラムtijouhaにセット

In [22]:
train.head(50)

,id,y,year,stage,match,time,home,away,stadium,tv,month,day,week
0,13994,18250,2012,1,1,14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合,3,10,土
1,13995,24316,2012,1,1,14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋,3,10,土
2,13996,17066,2012,1,1,14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪,3,10,土
3,13997,29603,2012,1,1,14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,スカパー／ｅ２／スカパー光／ＮＨＫ広島,3,10,土
4,13998,25353,2012,1,1,14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,スカパー／ｅ２／スカパー光（スカイ・Ａ ｓｐｏｒｔｓ＋）／ＮＨＫ札幌,3,10,土
5,13999,11283,2012,1,1,14:05,サガン鳥栖,セレッソ大阪,ベストアメニティスタジアム,スカパー／ｅ２／スカパー光／ＮＨＫ佐賀,3,10,土
6,14000,18920,2012,1,1,17:05,川崎フロンターレ,アルビレックス新潟,等々力陸上競技場,スカパー／ｅ２／スカパー光,3,10,土
7,14001,11437,2012,1,1,19:04,大宮アルディージャ,ＦＣ東京,ＮＡＣＫ５スタジアム大宮,スカパー／ｅ２／スカパー光／ＮＨＫ ＢＳ１,3,10,土
8,14002,13082,2012,1,1,14:04,柏レイソル,横浜Ｆ・マリノス,日立柏サッカー場,スカパー／ｅ２／スカパー光（ＴＢＳチャンネル）,3,11,日
9,14004,41069,2012,1,2,14:04,浦和レッズ,柏レイソル,埼玉スタジアム２００２,スカパー／ｅ２／スカパー光／ＮＨＫ総合,3,17,土


In [23]:
def Is_tijouha(a):
    for x in a:
        if "スカパー" in x:
            continue
        elif "２" in x:
            continue
        else:
            return 1
    
    return 0
    

In [24]:
train['tijouha'] = train['tv'].apply(lambda x: Is_tijouha(x.split('／')))

In [25]:
test['tijouha'] = test['tv'].apply(lambda x: Is_tijouha(x.split('／')))

In [26]:
# TV情報はドロップ
train = train.drop('tv', axis=1)
test = test.drop('tv', axis=1)

## キックオフ時刻を時間帯Hourに置き換え

In [27]:
time = train['time'].apply(lambda x: x.split(':')[0])
time = time.astype(int)
train['time'] = time

In [28]:
time = test['time'].apply(lambda x: x.split(':')[0])
time = time.astype(int)
test['time'] = time

## 週中の祝日カラムを調整

In [29]:
def split_week(x):
    if len(x) > 1:
        return x[0]
    else:
        return x

train['week'] = train['week'].apply(lambda x: split_week(x))
test['week'] = test['week'].apply(lambda x: split_week(x))

## 型調整

In [30]:
train['stage'] = train['stage'].astype(int)
test['stage'] = test['stage'].astype(int)

In [31]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1721 entries, 0 to 1720
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       1721 non-null   int64 
 1   y        1721 non-null   int64 
 2   year     1721 non-null   int64 
 3   stage    1721 non-null   int64 
 4   match    1721 non-null   int64 
 5   time     1721 non-null   int64 
 6   home     1721 non-null   object
 7   away     1721 non-null   object
 8   stadium  1721 non-null   object
 9   month    1721 non-null   int64 
 10  day      1721 non-null   int64 
 11  week     1721 non-null   object
 12  tijouha  1721 non-null   int64 
dtypes: int64(9), object(4)
memory usage: 174.9+ KB


### スタジアムデータを読み込み、マージする

In [32]:
stadium_data = pd.read_csv('stadium.csv')

In [33]:
stadium_data.head()

,name,address,capa
0,名古屋市瑞穂陸上競技場,愛知県名古屋市瑞穂区山下通5-1,20000
1,豊田スタジアム,愛知県豊田市千石町7-2,40000
2,フクダ電子アリーナ,千葉県千葉市中央区川崎町1-20,18500
3,日立柏サッカー場,千葉県柏市日立台1-2-50,15349
4,ニンジニアスタジアム,愛媛県松山市上野町乙46,15576


In [34]:
stadium_data.isnull().count()

name       59
address    59
capa       59
dtype: int64

In [35]:
train = train.merge(stadium_data, left_on='stadium', right_on='name', how='inner')
test = test.merge(stadium_data, left_on='stadium', right_on='name', how='inner')

In [36]:
train.head()

,id,y,year,stage,match,time,home,away,stadium,month,day,week,tijouha,name,address,capa
0,13994,18250,2012,1,1,14,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,3,10,土,1,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
1,14012,11831,2012,1,3,13,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,3,24,土,0,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
2,14031,13147,2012,1,5,14,ベガルタ仙台,ジュビロ磐田,ユアテックスタジアム仙台,4,7,土,1,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
3,14055,17192,2012,1,7,19,ベガルタ仙台,ＦＣ東京,ユアテックスタジアム仙台,4,21,土,1,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
4,14075,18525,2012,1,10,13,ベガルタ仙台,清水エスパルス,ユアテックスタジアム仙台,5,6,日,1,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694


In [37]:
test.head()

,id,year,stage,match,time,home,away,stadium,month,day,week,tijouha,name,address,capa
0,15822,2014,1,18,19,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,8,2,土,0,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
1,15840,2014,1,20,19,ベガルタ仙台,清水エスパルス,ユアテックスタジアム仙台,8,16,土,0,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
2,15863,2014,1,22,19,ベガルタ仙台,横浜Ｆ・マリノス,ユアテックスタジアム仙台,8,30,土,0,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
3,15885,2014,1,25,14,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,9,23,火,0,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
4,15903,2014,1,27,15,ベガルタ仙台,ＦＣ東京,ユアテックスタジアム仙台,10,5,日,1,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694


In [38]:
# 住所は削除
train = train.drop('address', axis=1)
test = test.drop('address', axis=1)

In [39]:
train.head()

,id,y,year,stage,match,time,home,away,stadium,month,day,week,tijouha,name,capa
0,13994,18250,2012,1,1,14,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,3,10,土,1,ユアテックスタジアム仙台,19694
1,14012,11831,2012,1,3,13,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,3,24,土,0,ユアテックスタジアム仙台,19694
2,14031,13147,2012,1,5,14,ベガルタ仙台,ジュビロ磐田,ユアテックスタジアム仙台,4,7,土,1,ユアテックスタジアム仙台,19694
3,14055,17192,2012,1,7,19,ベガルタ仙台,ＦＣ東京,ユアテックスタジアム仙台,4,21,土,1,ユアテックスタジアム仙台,19694
4,14075,18525,2012,1,10,13,ベガルタ仙台,清水エスパルス,ユアテックスタジアム仙台,5,6,日,1,ユアテックスタジアム仙台,19694


### 不要カラムの削除

In [40]:
train = train.drop(['stadium', 'name'], axis=1)
test = test.drop(['stadium', 'name'], axis=1)

In [41]:
train = train.drop('home', axis=1)
test = test.drop('home', axis=1)
train = train.drop('away', axis=1)
test = test.drop('away', axis=1)

##  ダミー変数

In [42]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [43]:
train.head(2)

,id,y,year,stage,match,time,month,day,tijouha,capa,week_土,week_日,week_月,week_木,week_水,week_火,week_金
0,13994,18250,2012,1,1,14,3,10,1,19694,1,0,0,0,0,0,0
1,14012,11831,2012,1,3,13,3,24,0,19694,1,0,0,0,0,0,0


In [44]:
test['week_木'] = 0
test.head(2)

,id,year,stage,match,time,month,day,tijouha,capa,week_土,week_日,week_月,week_水,week_火,week_金,week_木
0,15822,2014,1,18,19,8,2,0,19694,1,0,0,0,0,0,0
1,15840,2014,1,20,19,8,16,0,19694,1,0,0,0,0,0,0


## モデル作成、予測

In [45]:
train.head(2)

,id,y,year,stage,match,time,month,day,tijouha,capa,week_土,week_日,week_月,week_木,week_水,week_火,week_金
0,13994,18250,2012,1,1,14,3,10,1,19694,1,0,0,0,0,0,0
1,14012,11831,2012,1,3,13,3,24,0,19694,1,0,0,0,0,0,0


In [46]:
from sklearn.linear_model import LinearRegression as LR

In [47]:
model = LR()

In [48]:
y = train['y']
train = train.drop('y', axis=1)
y.head(5)

0    18250
1    11831
2    13147
3    17192
4    18525
Name: y, dtype: int64

In [49]:
trainX = train.copy()

In [50]:
model.fit(trainX, y)

LinearRegression()

In [51]:
testX = test.copy()
pred = model.predict(testX)

In [52]:
sample[1] = pred

In [53]:
sample.to_csv('submit2.csv', header=None, index=None)